In [2]:
import tensorflow as tf
import os
from sklearn.model_selection import train_test_split

In [4]:
# Define the path to the dataset
dataset_path = './testing'

# Create a list of file paths to the images
image_paths = [os.path.join(dataset_path, file) for file in os.listdir(dataset_path)]

In [5]:
image_paths

['./testing\\heatmap_0.png',
 './testing\\heatmap_1.png',
 './testing\\heatmap_2.png',
 './testing\\heatmap_3.png',
 './testing\\heatmap_4.png',
 './testing\\heatmap_5.png']

In [6]:
# Create a list of labels for each image
labels = [int(os.path.splitext(os.path.basename(path))[0].split('_')[-1]) for path in image_paths]

In [7]:
# Define the model architecture
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(500,500, 3)),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(10, activation='softmax')
])

In [8]:
# Compile the model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

Preprocessing

In [9]:
# Create a function to load and preprocess an image
def load_and_preprocess_image(path, label):
    # Load the image
    image = tf.io.read_file(path)
    image = tf.image.decode_jpeg(image, channels=3)
    # Preprocess the image
    image = tf.image.resize(image, (500, 500))
    image /= 255.0
    return image, label

splitting

In [10]:
# Split the data into training and validation sets
train_paths, val_paths, train_labels, val_labels = train_test_split(image_paths, labels, test_size=0.2,random_state=42)

In [11]:
train_paths, val_paths, train_labels, val_labels

(['./testing\\heatmap_5.png',
  './testing\\heatmap_2.png',
  './testing\\heatmap_4.png',
  './testing\\heatmap_3.png'],
 ['./testing\\heatmap_0.png', './testing\\heatmap_1.png'],
 [5, 2, 4, 3],
 [0, 1])

In [12]:
# Use the `tf.data` API to create datasets from the data
train_dataset = tf.data.Dataset.from_tensor_slices((train_paths, train_labels))
val_dataset = tf.data.Dataset.from_tensor_slices((val_paths, val_labels))

In [13]:
# Map the `load_and_preprocess_image` function over each element in the dataset
train_dataset = train_dataset.map(load_and_preprocess_image)
val_dataset = val_dataset.map(load_and_preprocess_image)

# Batch the data
train_dataset = train_dataset.batch(32)
val_dataset = val_dataset.batch(32)

# New Code

In [19]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Load the data
train_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
        './resm/train/',
        target_size=(150, 150),
        batch_size=32,
        class_mode='binary')

validation_generator = test_datagen.flow_from_directory(
        './resm/test/',
        target_size=(150, 150),
        batch_size=32,
        class_mode='binary')

# Define the model architecture
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(150, 150, 3)),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Conv2D(128, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

# Compile the model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train the model
history = model.fit_generator(
      train_generator,
      steps_per_epoch=100,
      epochs=15,
      validation_data=validation_generator,
      validation_steps=50)

# Evaluate the model
_, acc = model.evaluate_generator(validation_generator, steps=50)
print("Accuracy: ", acc)

Found 0 images belonging to 0 classes.
Found 0 images belonging to 0 classes.


C:\Users\Admin\AppData\Local\Temp\ipykernel_9440\3053604605.py:37: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  history = model.fit_generator(


ValueError: Asked to retrieve element 0, but the Sequence has length 0